### Import Dependencies

In [5]:
#%pip install pandas sqlalchemy pymysql python-dotenv
!pip install mysql.connector

  Using cached mysql-connector-2.2.9.tar.gz (11.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp311-cp311-win_amd64.whl size=248051 sha256=82b50e5ddcaba73ccdffae39cfc4d9a4f277465e49829bbd63372fefaf75688c
  Stored in directory: c:\users\ckart\appdata\local\pip\cache\wheels\17\cd\ed\2d49e9bac69cf09382e4c7cc20a2511202b48324b87db26019
Successfully built mysql.connector


In [6]:
import os
import pandas as pd
from sqlalchemy import create_engine
import altair as alt
import numpy as np
import mysql.connector
from sklearn.preprocessing import MultiLabelBinarizer
from dotenv import load_dotenv

alt.renderers.enable("default") 

RendererRegistry.enable('default')

In [26]:
import umap

### Omit Warnings

In [22]:
import warnings
warnings.filterwarnings(
    "ignore",
    message=".*convert_dtype parameter is deprecated.*"
)

# Exploratory Data Analysis

In [9]:
load_dotenv()

host = os.getenv("MYSQL_HOST", "localhost")
port = os.getenv("MYSQL_PORT", "3306")
user = os.getenv("MYSQL_USER","root")
password = os.getenv("MYSQL_PASSWORD")
db = os.getenv("MYSQL_DB", "rxnorm")

engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8mb4"
)


## Table Sizes

In [10]:
counts = pd.read_sql("""
SELECT 'RXNCONSO' AS Table_Name, COUNT(*) AS `Rows` FROM rxnconso
UNION ALL
SELECT 'RXNREL'   AS Table_Name, COUNT(*) AS `Rows` FROM rxnrel
UNION ALL
SELECT 'RXNSAT'   AS Table_Name, COUNT(*) AS `Rows` FROM rxnsat;
""", engine)

bars = alt.Chart(counts).mark_bar().encode(
    x=alt.X("Table_Name:N", sort="-y", title="Table Name"),
    y=alt.Y("Rows:Q",
            title="Number of Rows",
            scale=alt.Scale(domain=[0, counts["Rows"].max()*1.1])),
    tooltip=["Table_Name", "Rows"]
)

# Text labels on top
text = alt.Chart(counts).mark_text(
    dy=-5  # moves text slightly above bar
).encode(
    x=alt.X("Table_Name:N", sort="-y"),
    y="Rows:Q",
    text=alt.Text("Rows:Q", format=",")
)

chart = (bars + text).properties(
    title="Row Distribution Across RxNorm Tables",
    width=500,
    height=400
)

chart

alt.LayerChart(...)

## Table Structures

In [11]:
rxnconso = pd.read_sql("""
SELECT * 
FROM RXNCONSO LIMIT 25
""", engine)

rxnrel = pd.read_sql("""
SELECT * 
FROM RXNREL LIMIT 25
""", engine)

rxnsat = pd.read_sql("""
SELECT * 
FROM RXNSAT LIMIT 25
""", engine)

In [12]:
rxnconso.head(10)

,RXCUI,LAT,TS,LUI,STT,SUI,ISPREF,RXAUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,38,ENG,,,,,,829,829,38,,RXNORM,BN,38,Parlodel,,N,4096
1,44,ENG,,,,,,12251526,12251526,44,,RXNORM,IN,44,mesna,,N,4096
2,44,ENG,,,,,,2798745,2798745,44,,MTHSPL,SU,NR7O1405Q9,mesna,,N,4096
3,44,ENG,,,,,,2982613,2982613,44,,MTHSPL,SU,NR7O1405Q9,MESNA,,N,4096
4,61,ENG,,,,,,12254378,12254378,61,,RXNORM,IN,61,beta-alanine,,N,4096
5,61,ENG,,,,,,9183401,9183401,61,,MTHSPL,SU,11P2JDE17B,.BETA.-ALANINE,,N,4096
6,73,ENG,,,,,,12252051,12252051,73,,RXNORM,IN,73,docosahexaenoate,,N,4096
7,74,ENG,,,,,,12253228,12253228,74,,RXNORM,IN,74,4-aminobenzoic acid,,N,4096
8,74,ENG,,,,,,3303230,3303230,74,,MTHSPL,SU,TL2TJE8QTX,AMINOBENZOIC ACID,,N,4096
9,74,ENG,,,,,,4230758,4230758,74,,MTHSPL,SU,TL2TJE8QTX,Aminobenzoic Acid,,N,4096


The RXNCONSO table is the concept dictionary within the RxNorm data model. It contains all normalized drug concepts represented by unique identifiers known as RXCUIs (RxNorm Concept Unique Identifiers). Each row in this table corresponds to a specific lexical representation (string) associated with a drug concept. However, a single drug concept may appear across multiple rows due to the presence of multiple naming variations, source vocabularies, and semantic classifications. The table includes attributes such as language (LAT), term type (TTY), source abbreviation (SAB), suppression status (SUPPRESS), and the textual string representation (STR). The term type field is particularly critical, as it categorizes each concept into semantic classes such as Ingredient (IN), Brand Name (BN), Semantic Clinical Drug (SCD), Semantic Branded Drug (SBD), Synonym (SY), Precise Ingredient (PIN), and others. This classification enables structured filtering and semantic analysis of drug entities at different levels of specificity. RXNCONSO therefore functions as the primary lookup table for identifying drugs, distinguishing between brand and generic forms, and retrieving standardized clinical representations. In analytical workflows, this table is used to extract ingredient-level concepts, identify branded products, and construct clean drug dictionaries for downstream modeling and dashboard visualization.

In [13]:
rxnrel.head(10)

,RXCUI1,RXAUI1,STYPE1,REL,RXCUI2,RXAUI2,STYPE2,RELA,RUI,SRUI,SAB,SL,DIR,RG,SUPPRESS,CVF
0,,710074,AUI,SY,,12366353,AUI,,,,RXNORM,,,,,4096
1,,710393,AUI,SY,,12366828,AUI,,,,RXNORM,,,,,4096
2,,710397,AUI,SY,,12359611,AUI,,,,RXNORM,,,,,4096
3,,710567,AUI,SY,,12365485,AUI,,,,RXNORM,,,,,4096
4,,711490,AUI,SY,,12363086,AUI,,,,RXNORM,,,,,4096
5,,713495,AUI,SY,,12367781,AUI,,,,RXNORM,,,,,4096
6,,713677,AUI,SY,,12363087,AUI,,,,RXNORM,,,,,4096
7,,713681,AUI,SY,,12363071,AUI,,,,RXNORM,,,,,4096
8,,713685,AUI,SY,,12359356,AUI,,,,RXNORM,,,,,4096
9,,713689,AUI,SY,,12363075,AUI,,,,RXNORM,,,,,4096


The RXNREL table encodes the relationships between drug concepts defined in RXNCONSO, effectively forming the relational backbone of the RxNorm knowledge graph. Each row represents a directed relationship between two RXCUIs, specified by the fields RXCUI1 and RXCUI2. These identifiers correspond to drug entities that are semantically connected through relationship types defined by the REL and RELA attributes. While REL provides broad relationship categories (e.g., parent-child), the RELA field specifies detailed semantic connections such as "has_ingredient," "ingredient_of," "tradename_of," "has_form," and "has_strength." Through these structured relationships, RXNREL enables reconstruction of complex drug hierarchies, including mappings between brand names and generic ingredients, identification of combination drugs, and decomposition of clinical drug representations into constituent components. This relational structure transforms the dataset from a simple lookup table into a structured biomedical knowledge network. For analytical and dashboard purposes, RXNREL is critical for identifying which brand drugs share the same active ingredient, determining ingredient overlap across products, and quantifying the structural similarity between pharmaceutical formulations. Thus, RXNREL supports advanced semantic querying and graph-based analysis of pharmaceutical entities.

In [14]:
rxnsat.head(10)

,RXCUI,LUI,SUI,RXAUI,STYPE,CODE,ATUI,SATUI,ATN,SAB,ATV,SUPPRESS,CVF
0,38,,,829,AUI,38,,,RXN_BN_CARDINALITY,RXNORM,single,N,4096
1,44,,,2798745,AUI,NR7O1405Q9,,,SPL_SET_ID,MTHSPL,2a2a526f-636b-7920-4261-6c626f612a2a,N,4096
2,44,,,2982613,AUI,NR7O1405Q9,,,SPL_SET_ID,MTHSPL,1dd69e84-616d-fb77-e063-6394a90af2f4,N,4096
3,44,,,2982613,AUI,NR7O1405Q9,,,SPL_SET_ID,MTHSPL,2e8eebc3-ea75-4c57-bfe1-2c5dc2c37806,N,4096
4,44,,,2982613,AUI,NR7O1405Q9,,,SPL_SET_ID,MTHSPL,49fdfbc3-69f8-4706-9d11-bf60e4e7811b,N,4096
5,44,,,2982613,AUI,NR7O1405Q9,,,SPL_SET_ID,MTHSPL,4a51ffdb-f59f-4e02-b3c4-f3457ab7b514,N,4096
6,44,,,2982613,AUI,NR7O1405Q9,,,SPL_SET_ID,MTHSPL,8012b97b-4e39-451f-8e04-542d196e8bc6,N,4096
7,44,,,2982613,AUI,NR7O1405Q9,,,SPL_SET_ID,MTHSPL,90d64539-33ad-47e8-9de7-17d6c5baaba4,N,4096
8,44,,,2982613,AUI,NR7O1405Q9,,,SPL_SET_ID,MTHSPL,a7af968c-05cf-4438-94ba-96a2baf9c367,N,4096
9,44,,,2982613,AUI,NR7O1405Q9,,,SPL_SET_ID,MTHSPL,a880160b-fa82-453a-a971-29ab897876a7,N,4096


The RXNSAT table provides supplemental attribute-level metadata for drug concepts defined in RXNCONSO. Unlike RXNCONSO, which focuses on naming and classification, and RXNREL, which encodes semantic relationships, RXNSAT stores additional descriptive attributes associated with individual RXCUIs. Each row links an RXCUI to a specific attribute name (ATN) and attribute value (ATV), along with source information (SAB) and optional suppression indicators. Attributes may include National Drug Codes (NDC), dosage strength information, external vocabulary mappings, regulatory identifiers, formulation characteristics, and other clinically relevant metadata. This structure allows enrichment of core drug entities with regulatory and descriptive properties that extend beyond conceptual relationships. RXNSAT is particularly important when integrating RxNorm data with external systems, performing regulatory analysis, or constructing dashboards that require dosage or packaging information. Because this table contains millions of records, it reflects the extensive attribute-level detail embedded within RxNorm. In analytical workflows, RXNSAT enables retrieval of dosage strengths, external identifiers, and mapping information necessary for granular pharmaceutical comparison. Together with RXNCONSO and RXNREL, RXNSAT completes the tripartite architecture of RxNorm by adding descriptive depth to the concept and relationship layers.

In [15]:
tty_counts = pd.read_sql("""
SELECT tty AS Terms, COUNT(*) AS Count
FROM rxnconso
WHERE sab='RXNORM' AND suppress='N'
GROUP BY tty
ORDER BY Count DESC;
""", engine)

bars = alt.Chart(tty_counts).mark_bar().encode(
    x=alt.X("Terms:N", sort="-y", title="Term Type (TTY)"),
    y=alt.Y(
        "Count:Q",
        title="Number of Concepts",
        scale=alt.Scale(domain=[0, tty_counts["Count"].max()*1.1])
    ),
    tooltip=["Terms", "Count"]
)

# Text labels
text = alt.Chart(tty_counts).mark_text(
    dy=-5,size=8
).encode(
    x=alt.X("Terms:N", sort="-y"),
    y="Count:Q",
    text=alt.Text("Count:Q", format=",")
)

chart = (bars + text).properties(
    title="Distribution of RxNorm Term Types (TTY)",
    width=600,
    height=400
)

chart

alt.LayerChart(...)

The above chart shows the distribution of all the terms persent in the RXNCOSCO table. From this we can see that there are in total 5,825 Ingredients, 4,187 Brand Names, 1,923 Precise Ingredients, etc.

In [16]:
alt.data_transformers.disable_max_rows()

rel = pd.read_sql("""
SELECT RXCUI1 AS drug_rxcui,
       RXCUI2 AS ingredient_rxcui
FROM rxnrel
WHERE RELA = 'has_ingredient'
LIMIT 500000;
""", engine)

ingredient_count = (
    rel.groupby("drug_rxcui")
       .size()
       .reset_index(name="n_ingredients")
)

cap = int(ingredient_count["n_ingredients"].quantile(0.95))
ingredient_count["n_ingredients_capped"] = ingredient_count["n_ingredients"].clip(upper=cap)

hist = alt.Chart(ingredient_count).mark_bar(
    cornerRadiusTopLeft=6,
    cornerRadiusTopRight=6
).encode(
    x=alt.X(
        "n_ingredients_capped:Q",
        bin=alt.Bin(maxbins=25),
        title=f"Number of Ingredients per Drug (capped at {cap})"
    ),
    y=alt.Y("count():Q", title="Number of Drugs"),
    tooltip=[
        alt.Tooltip("count():Q", title="Drugs", format=","),
        alt.Tooltip("n_ingredients_capped:Q", bin=alt.Bin(maxbins=25), title="Ingredient bin")
    ]
).properties(
    title="Distribution of Ingredient Count per Drug",
    width=750,
    height=420
).interactive()

hist


alt.Chart(...)

This is an Example of Intraplot with tootltips.

### Most Common Drugs

In [17]:
ingredient_freq = (
    rel.groupby("ingredient_rxcui")
       .size()
       .reset_index(name="freq")
       .sort_values("freq", ascending=False)
       .head(30)
)
chart = alt.Chart(ingredient_freq).mark_bar(
    cornerRadiusTopLeft=0,
    cornerRadiusTopRight=0
).encode(
    x=alt.X(
        "ingredient_rxcui:N",
        sort="-y",
        title="Ingredient RXCUI"
    ),
    y=alt.Y(
        "freq:Q",
        scale=alt.Scale(domain=[0, 28]),
        title="Frequency"
    ),
    tooltip=["ingredient_rxcui", "freq"]
).properties(
    title="Top 30 Most Used Ingredients (by Drug Count)",
    width=750,
    height=420
)


chart

alt.Chart(...)

## Interplot

In [18]:
drug_names = ["Tylenol", "Parlodel", "Bonine", "Anafranil", "Adipex-P", "ATGAM"]
drug_in = ", ".join([f"'{d}'" for d in drug_names])

drugs_pref = pd.read_sql(f"""
SELECT rxcui AS drug_rxcui, str AS drug_name, tty
FROM rxnconso
WHERE sab='RXNORM' AND suppress='N'
  AND str IN ({drug_in})
  AND tty IN ('SBD','SCD','SBDC','SCDC','SBDG','SCDG','BN','PSN')
ORDER BY
  FIELD(tty,'SBD','SCD','SBDC','SCDC','SBDG','SCDG','PSN','BN');
""", engine)

# keep best tty per drug_name (top row after sorting)
drugs_best = drugs_pref.drop_duplicates("drug_name", keep="first").reset_index(drop=True)
drugs_best


,drug_rxcui,drug_name,tty
0,38,Parlodel,BN
1,332,Adipex-P,BN
2,756,Anafranil,BN
3,1204,ATGAM,BN
4,1694,Bonine,BN
5,202433,Tylenol,BN


In [19]:
drug_list_in = ", ".join(map(str, drugs_best["drug_rxcui"].tolist()))

drug_ing = pd.read_sql(f"""
SELECT r.rxcui1 AS drug_rxcui,
       r.rxcui2 AS ingredient_rxcui
FROM rxnrel r
WHERE r.rela='has_ingredient'
  AND r.rxcui1 IN ({drug_list_in});
""", engine)

# name-map fallback (works even when TTY isn't IN)
ingredient_names = pd.read_sql("""
SELECT rxcui AS ingredient_rxcui,
       COALESCE(
         MAX(CASE WHEN tty='IN'  AND sab='RXNORM' AND suppress='N' THEN str END),
         MAX(CASE WHEN sab='RXNORM' AND suppress='N' THEN str END),
         MAX(str)
       ) AS ingredient_name
FROM rxnconso
GROUP BY rxcui;
""", engine)

ingredient_global = pd.read_sql("""
SELECT rxcui2 AS ingredient_rxcui,
       COUNT(DISTINCT rxcui1) AS global_drug_count
FROM rxnrel
WHERE rela='has_ingredient'
GROUP BY rxcui2;
""", engine)

df2 = (drug_ing
       .merge(drugs_best[["drug_rxcui","drug_name"]], on="drug_rxcui", how="left")
       .merge(ingredient_names, on="ingredient_rxcui", how="left")
       .merge(ingredient_global, on="ingredient_rxcui", how="left"))

df2[["drug_name","ingredient_rxcui","ingredient_name","global_drug_count"]].head()


,drug_name,ingredient_rxcui,ingredient_name,global_drug_count
0,Parlodel,105050,Parlodel 5 MG Oral Capsule,1
1,Parlodel,105446,Parlodel SnapTabs 2.5 MG Oral Tablet,1
2,Parlodel,366490,bromocriptine Oral Capsule [Parlodel],1
3,Parlodel,368543,bromocriptine Oral Tablet [Parlodel],1
4,Parlodel,563894,bromocriptine 5 MG [Parlodel],1


In [24]:
import altair as alt
alt.renderers.enable("default")

drug_summary = (df2.groupby("drug_name", as_index=False)
                  .agg(n_ingredients=("ingredient_rxcui","nunique"))
                  .sort_values("n_ingredients", ascending=False))

drug_ingredients = (df2.dropna(subset=["ingredient_name"])
                      .groupby(["drug_name","ingredient_name"], as_index=False)
                      .agg(global_drug_count=("global_drug_count","max")))

sel = alt.selection_point(fields=["drug_name"], on="click", clear=False)

plot1 = alt.Chart(drug_summary).mark_bar().encode(
    x=alt.X("drug_name:N", sort="-y", title="Drug"),
    y=alt.Y("n_ingredients:Q", title="No of Products", scale=alt.Scale(domainMin=0)),
    tooltip=["drug_name","n_ingredients"],
    opacity=alt.condition(sel, alt.value(1), alt.value(0.25))
).add_params(sel).properties(width=700, height=260, title="Plot 1: Select the Drug")

plot2 = alt.Chart(drug_ingredients).transform_filter(sel).mark_bar().encode(
    x=alt.X("ingredient_name:N", sort="-y", title="Products (Selected Drug)"),
    y=alt.Y("global_drug_count:Q", scale=alt.Scale(domainMin=0)),
    tooltip=["ingredient_name", alt.Tooltip("global_drug_count:Q", format=",")]
).properties(width=700, height=320, title="Plot 2: Products of Selected Drug")

(plot1 & plot2).resolve_scale(x="independent")


alt.VConcatChart(...)

## Cluster Scatter Plot

In [25]:
QUERY_SMALL = """
SELECT
  r.RXCUI1 AS drug_rxcui,
  d.STR AS drug_name,
  r.RXCUI2 AS ing_rxcui,
  i.STR AS ing_name
FROM rxnrel r
JOIN rxnconso d ON d.RXCUI = r.RXCUI1
JOIN rxnconso i ON i.RXCUI = r.RXCUI2
WHERE r.RELA = 'has_ingredient'
LIMIT 2000
"""

# 3) Read SQL with engine (no mysql.connector needed)
pairs = pd.read_sql(QUERY_SMALL, con=engine)

print("Rows:", len(pairs))
print(pairs.head())

drug_to_ings = (
    pairs.groupby(["drug_rxcui", "drug_name"])["ing_name"]
    .apply(lambda x: list(set(x)))
    .reset_index(name="ingredients")
)

drug_to_ings = drug_to_ings.head(200)
print("Drugs used:", len(drug_to_ings))

mlb = MultiLabelBinarizer()
X = mlb.fit_transform(drug_to_ings["ingredients"])
print("Matrix shape:", X.shape)

reducer = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    metric="jaccard",
    random_state=42,
    init="random"   # avoid spectral warning
)

embedding = reducer.fit_transform(X)

plot_df = drug_to_ings[["drug_rxcui", "drug_name"]].copy()
plot_df["x"] = embedding[:, 0]
plot_df["y"] = embedding[:, 1]

# (Optional) make tooltips nicer
chart = (
    alt.Chart(plot_df)
    .mark_circle(size=60)
    .encode(
        x=alt.X("x:Q", title="UMAP-1"),
        y=alt.Y("y:Q", title="UMAP-2"),
        tooltip=[
            alt.Tooltip("drug_name:N", title="Drug"),
            alt.Tooltip("drug_rxcui:O", title="RXCUI"),
        ],
    )
    .properties(
        title="Drug Ingredient Similarity (Small Sample)",
        width=650,
        height=450
    )
    .interactive()
)

chart

Rows: 2000
  drug_rxcui drug_name ing_rxcui                                     ing_name
0         38  Parlodel    105050   bromocriptine 5 MG Oral Capsule [Parlodel]
1         38  Parlodel    105050                   Parlodel 5 MG Oral Capsule
2         38  Parlodel    105050                   Parlodel 5 MG Oral Capsule
3         38  Parlodel    105446  bromocriptine 2.5 MG Oral Tablet [Parlodel]
4         38  Parlodel    105446         Parlodel SnapTabs 2.5 MG Oral Tablet
Drugs used: 76
Matrix shape: (76, 545)


C:\Users\ckart\miniconda3\envs\rxviz\Lib\site-packages\umap\umap_.py:1887: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  warn(
C:\Users\ckart\miniconda3\envs\rxviz\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
C:\Users\ckart\miniconda3\envs\rxviz\Lib\site-packages\umap\umap_.py:127: UserWarning: A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 5536 edges.
It has only fully disconnected 2 vertices.
Use umap.utils.disconnected_vertices() to identify them.
  warn(


alt.Chart(...)